In [1]:
import tensorflow as tf
import glob
#import tensorflow_io as tfio
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy import signal
import matplotlib
matplotlib.use('Agg')

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True

sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



In [3]:
tf.__version__

'2.3.0'

In [4]:
tf.test.is_gpu_available()


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [8]:
im = Image.open('UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/Train001/002.tif')
im = im.resize((230,150))
print(np.array(im).shape)
plt.imshow(im)

(150, 230)


In [9]:
IMG_HEIGHT=100
IMG_WIDTH=100
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE=16

train_set = 'UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/*/*.tif'

train_files = sorted(glob.glob(train_set))

a = np.zeros((len(train_files),IMG_HEIGHT, IMG_WIDTH,1)).astype(np.float32)

for idx, filename in enumerate(train_files):
    im = Image.open(filename)
    im = im.resize((IMG_WIDTH, IMG_HEIGHT))
    a[idx,:,:,0] = np.array(im)/255.0
    

print(a.shape)

train_dataset  = tf.data.Dataset.from_tensor_slices((a, a)).shuffle(1000, 
                                                                    seed=42, 
                                                                    reshuffle_each_iteration=False).batch(BATCH_SIZE)
print(len(list(train_dataset)))

(6800, 100, 100, 1)
3400


In [10]:
test_set = 'UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test024/*'

test_files = sorted(glob.glob(test_set))

a = np.zeros((len(test_files),IMG_WIDTH, IMG_HEIGHT,1)).astype(np.float32)

for idx,filename in enumerate(test_files):
    im = Image.open(filename)
    im = im.resize((IMG_WIDTH,IMG_HEIGHT))
    a[idx,:,:,0] = np.array(im, dtype=np.float32)/255.0

    
test_dataset = tf.data.Dataset.from_tensor_slices((a, a)).batch(1)
print(len(list(test_dataset)))

200


In [11]:
count = 0
for x,y in train_dataset.take(BATCH_SIZE):
    print(x.shape, y.shape)
    count = count + 1
    if count > 5:
        break

(2, 100, 100, 1) (2, 100, 100, 1)
(2, 100, 100, 1) (2, 100, 100, 1)


In [12]:
import matplotlib.pyplot as plt

for image,label in train_dataset.take(1):
    print(image.shape)
    img = image[0][:,:,0]
    lbl = label[0][:,:,0]
    plt.imshow(img,cmap=plt.cm.gray, interpolation='nearest')
    plt.show()
    plt.imshow(lbl,cmap=plt.cm.gray, interpolation='nearest')
    plt.show()
    print(img)

(2, 100, 100, 1)
tf.Tensor(
[[0.38039216 0.40392157 0.49019608 ... 0.35686275 0.23921569 0.27058825]
 [0.34509805 0.41568628 0.48235294 ... 0.34901962 0.3019608  0.2784314 ]
 [0.4        0.34117648 0.34509805 ... 0.2901961  0.34117648 0.44705883]
 ...
 [0.45490196 0.45882353 0.46666667 ... 0.6509804  0.65882355 0.6901961 ]
 [0.47843137 0.49803922 0.5058824  ... 0.6745098  0.67058825 0.69411767]
 [0.5058824  0.52156866 0.53333336 ... 0.69803923 0.6745098  0.69803923]], shape=(100, 100), dtype=float32)


<ipython-input-12-c92bdae2075f>:8: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-12-c92bdae2075f>:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [13]:
#strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

#with strategy.scope():
conv_encoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(IMG_HEIGHT,IMG_WIDTH,1)),
                tf.keras.layers.Conv2D(32, 
                                       kernel_size=5, 
                                       activation='relu'
                                      ),
                tf.keras.layers.MaxPool2D(pool_size=2),
                tf.keras.layers.Conv2D(32, 
                                       kernel_size=5,
                                       activation='relu'
                                      ),
                tf.keras.layers.MaxPool2D(pool_size=2),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(1000)
            ]
        )

conv_decoder = tf.keras.Sequential(
            [   
                tf.keras.layers.Dense(22*22*32, 
                                      activation='relu'
                                     ),
                tf.keras.layers.Reshape(target_shape=(22,22,32)),
                tf.keras.layers.UpSampling2D(2, interpolation='nearest'),
                tf.keras.layers.Conv2DTranspose(32, 
                                                kernel_size=5, 
                                                activation="relu"
                                               ),
                tf.keras.layers.UpSampling2D(2, interpolation='nearest'),
                tf.keras.layers.Conv2DTranspose(1, 
                                                kernel_size=5, 
                                                activation="sigmoid"
                                               )
            ]            
        )


In [14]:
image_input = tf.keras.Input(shape=(IMG_HEIGHT,IMG_WIDTH,1))
encoded = conv_encoder(image_input)
print(encoded.shape)
decoded = conv_decoder(encoded)
print(decoded.shape)

(None, 1000)
(None, 100, 100, 1)


In [15]:
conv_ae = tf.keras.Model(inputs=image_input, outputs=decoded)

#conv_ae = tf.keras.Sequential([conv_encoder, conv_decoder])
conv_ae.compile(loss=tf.keras.losses.MeanSquaredError(), 
        optimizer=tf.keras.optimizers.Adam(lr=1e-4, decay=1e-4),
        metrics=['mae'])

In [16]:
#conv_ae(np.ones((1,100,100,1), dkaggletype=np.float32))

conv_ae.summary()




Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100, 100, 1)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 1000)              15515464  
_________________________________________________________________
sequential_1 (Sequential)    (None, 100, 100, 1)       15529921  
Total params: 31,045,385
Trainable params: 31,045,385
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = conv_ae.fit(train_dataset, epochs=30, validation_data=test_dataset)


Epoch 1/30
 678/3400 [====>.........................] - ETA: 33s - loss: 0.0075 - mae: 0.0515

KeyboardInterrupt: 

In [ ]:
def plot_training_loss(history):

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

In [ ]:
plot_training_loss(history)

In [ ]:
def plot_image(image):
    print('plot image {}'.format(image.shape))
    plt.imshow(image[:,:,0], cmap=plt.cm.gray, interpolation='nearest')
    plt.axis("off")

In [ ]:
def show_reconstructions(model, images, n_images=5):
    print('show reconstructions = {}'.format(images.shape))
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 3, 5))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])

In [ ]:
test_images = list(test_dataset.take(1))[0]
test_image = test_images[0]
print(test_image.shape)
show_reconstructions(conv_ae, test_image, n_images=1)

In [ ]:
def plot(img, output, diff, H, threshold, counter):
    #print('inside plot, imgshape {}'.format(img.shape))
    fig, (ax0, ax1, ax2,ax3) = plt.subplots(ncols=4, figsize=(10, 5))
    ax0.set_axis_off()
    ax1.set_axis_off()
    ax2.set_axis_off()
    
    ax0.set_title('input image')
    ax1.set_title('reconstructed image')
    ax2.set_title('diff ')
    ax3.set_title('anomalies')
    
    ax0.imshow(img, cmap=plt.cm.gray, interpolation='nearest') 
    ax1.imshow(output, cmap=plt.cm.gray, interpolation='nearest')   
    ax2.imshow(diff, cmap=plt.cm.viridis, vmin=0, vmax=255, interpolation='nearest')  
    ax3.imshow(img, cmap=plt.cm.gray, interpolation='nearest')
    
    x,y = np.where(H > threshold)
    ax3.scatter(y,x,color='red',s=0.1) 

    plt.axis('off')
    
    fig.savefig('images/{:0>3d}.png'.format(counter))

In [ ]:
threshold = 4*255
counter = 0;

for image, _  in test_dataset:
    counter = counter + 1
    output = conv_ae.predict(image)
    output = tf.multiply(output,255.)
    img = tf.multiply(tf.cast(image, tf.float32), 255.)
    diff = tf.subtract(output,img)
    tmp = diff[0,:,:,0]
    H = signal.convolve2d(tmp, np.ones((4,4)), mode='same')
    plot(img[0,:,:,0], output[0,:,:,0], diff[0,:,:,0], H, threshold, counter)
    

In [ ]:
from PIL import Image
import glob
 
# Create the frames
frames = []
imgs = sorted(glob.glob('images/*.png'))
print(imgs[:10])
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
 
# Save into a GIF file that loops forever
frames[0].save('result.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)